In [1]:
# importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import lxml

In [2]:
# web scrapping
web_url = requests.get('https://au.indeed.com/cmp/Deloitte/reviews?fcountry=ALL&start=', timeout=5)
print(web_url.text)

<!doctype html>
<html lang="en" dir="ltr">
<head>
    <title>Working at Deloitte: 10,047 Reviews | Indeed.com</title>
    <meta http-equiv="content-type" content="text/html;charset=UTF-8">
        <meta name="description" content="10,047 reviews from Deloitte employees about Deloitte culture, salaries, benefits, work-life balance, management, job security, and more.">
        <meta name="viewport" content="width=device-width, initial-scale=1">

        <link rel="canonical" href="https://au.indeed.com/cmp/Deloitte/reviews?fcountry&#61;ALL">
        <link rel="next" href="https://au.indeed.com/cmp/Deloitte/reviews?fcountry&#61;ALL&amp;start&#61;20">
            <link rel="alternate" href="https://ar.indeed.com/cmp/Deloitte/reviews?fcountry&#61;ALL" hreflang="es-AR">
            <link rel="alternate" href="https://au.indeed.com/cmp/Deloitte/reviews?fcountry&#61;ALL" hreflang="en-AU">
            <link rel="alternate" href="https://be.indeed.com/cmp/Deloitte/reviews?fcountry&#61;ALL" href

In [5]:
# creating a parse function
def parse(all_url):
    
    content = BeautifulSoup(all_url.content, 'lxml')
    respository = content.findAll('div',{'class':'cmp-Review-container'})
    df = pd.DataFrame(columns = ['rating', 'rating_title',  'rating_description','rating_pros', 'rating_cons'])
    
    for item in respository:        
        try:
            rating = item.find('', {'class': 'cmp-ReviewRating-text'}).text.replace('\n', '')
        except:
            rating = None
        
        try:
            rating_title = item.find('div',{'class': 'cmp-Review-title'}).text.replace('\n', '')
        except:
            rating_title = None
        
        try:
            rating_description = item.find('span',{'itemprop': 'reviewBody'}).text.replace('\r', '. ')
        except:
            rating_description = None
        
        try:
            rating_pros = item.find('div',{'class': 'cmp-ReviewProsCons-prosText'}).text.replace('\n', '')
        except:
            rating_pros = None
       
        try:
            rating_cons = item.find('div',{'class': 'cmp-ReviewProsCons-consText'}).text.replace('\n', '')
        except:
            rating_cons = None
            
        df = df.append({'rating': rating, 
             'rating_title': rating_title, 
             'rating_description': rating_description,
             'rating_pros': rating_pros, 
             'rating_cons': rating_cons}, ignore_index=True)
    return df

In [6]:
web_url = 'https://au.indeed.com/cmp/Deloitte/reviews?fcountry=ALL&start='

all_pages_df = pd.DataFrame(columns = ['rating', 'rating_title', 'rating_description','rating_pros', 'rating_cons'])

num_reviews = 20
# you can adjust this number on how many reviews you which to scrape
while num_reviews < 4000:  
    
    all_url = web_url + str(num_reviews)
    
    df_url = requests.get(all_url, timeout=5)  
        
    partial_pages_df = parse(df_url)   
    all_pages_df = all_pages_df.append(partial_pages_df, ignore_index=True) 
    
    num_reviews += 20

In [7]:
all_pages_df.to_csv('indeed_data.csv')


In [1]:
all_pages_df.head()

NameError: name 'all_pages_df' is not defined

all_pages_df.shape